## Construção base de municípios

### Dependências

In [181]:
import requests
from tqdm import tqdm
import pandas as pd
from geopy.geocoders import Nominatim
import numpy as np

### API IBGE - Dados gerais

Fonte: https://servicodados.ibge.gov.br/api/docs/localidades#api-Municipios-municipiosGet

In [169]:
response = requests.get('https://servicodados.ibge.gov.br/api/v1/localidades/municipios')

In [170]:
response.json()[0]

{'id': 1100015,
 'nome': "Alta Floresta D'Oeste",
 'microrregiao': {'id': 11006,
  'nome': 'Cacoal',
  'mesorregiao': {'id': 1102,
   'nome': 'Leste Rondoniense',
   'UF': {'id': 11,
    'sigla': 'RO',
    'nome': 'Rondônia',
    'regiao': {'id': 1, 'sigla': 'N', 'nome': 'Norte'}}}},
 'regiao-imediata': {'id': 110005,
  'nome': 'Cacoal',
  'regiao-intermediaria': {'id': 1102,
   'nome': 'Ji-Paraná',
   'UF': {'id': 11,
    'sigla': 'RO',
    'nome': 'Rondônia',
    'regiao': {'id': 1, 'sigla': 'N', 'nome': 'Norte'}}}}}

In [171]:
# Tabela com nome e id dos municípios
municipios = pd.DataFrame(response.json()).drop(columns=["microrregiao", "regiao-imediata"], axis=1)

municipios['id'] = municipios['id'].apply(lambda x : str(x))

# Ferramenta para geolocalização
geolocator = Nominatim(user_agent="municipios")

# Arrays para serem adicionados à tabela
uf_array       = []
uf_id_array    = []
uf_name_array  = []
regiao_array   = []
lat_array      = []
lng_array      = []

# Interando por município
for municipio in tqdm(response.json()):
    
    # Dados diretos da API
    uf_array.append(municipio['microrregiao']['mesorregiao']['UF']['sigla'])
    uf_id_array.append(municipio['microrregiao']['mesorregiao']['UF']['id'])
    uf_name_array.append(municipio['microrregiao']['mesorregiao']['UF']['nome'])
    regiao_array.append(municipio['microrregiao']['mesorregiao']['UF']['regiao']['nome'])
    
    # Georreferenciando municípios
    municipio_name = municipio['nome']
    address        = f'{municipio_name}, {uf_name_array[-1]}'
    
    location = geolocator.geocode(address)
    
    try:
        lat_array.append(location.latitude)
        lng_array.append(location.longitude)
    except:
        lat_array.append(None)
        lng_array.append(None)

# Salvando dados na tabela
municipios['uf_nome']   = uf_name_array
municipios['uf_id']     = uf_id_array
municipios['uf']        = uf_array
municipios['regiao']    = regiao_array
municipios['latitude']  = lat_array
municipios['longitude'] = lng_array

100%|██████████| 5570/5570 [46:28<00:00,  2.00it/s] 


### Estimativas de populacao - 2013 a 2017

Fonte: https://www.ibge.gov.br/estatisticas/sociais/populacao/9103-estimativas-de-populacao.html?=&t=downloads

In [172]:
def make_id(dataframe):
    
    id_array = []
    
    for index in dataframe.index:
        
        municio = dataframe.loc[index]
        
        
        uf_id  = municio['uf_id']
        mun_id = municio['mun_id']
        id_array.append(f'{uf_id}{mun_id}')
        
    return id_array

def preprocess_population(value):
    
    if isinstance(value, str):
        
        val = value.replace('.', '')
        
        if '(' in val:
            val = int(val.split('(')[0].strip())
            return val
        elif '*' in val:
            val = int(val.split('*')[0].strip())
            return val
    
    return int(value)

In [173]:
# Leitura das bases de populcacao

years = [2013, 2014, 2015, 2016, 2017]
population_tables = {}

for year in tqdm(years):
    
    if year in [2014, 2015, 2016, 2017]:
        df = pd.pandas.read_excel(f'./../data/populacao_IBGE/estimativa_{year}.xls', sheet_name=1).dropna(axis=0)
    else:
        df = pd.pandas.read_excel(f'./../data/populacao_IBGE/estimativa_{year}.xls').dropna(axis=0)
        
    if year in [2016]:
        df = df.drop([0], axis=0).rename(columns={'ESTIMATIVAS DA POPULAÇÃO RESIDENTE NOS MUNICÍPIOS BRASILEIROS COM DATA DE REFERÊNCIA EM 1º DE JULHO DE 2016': 'uf', 'Unnamed: 1': 'uf_id', 'Unnamed: 2': 'mun_id', 'Unnamed: 3': 'nome', 'Unnamed: 4': 'populacao'}).reset_index(drop=True)
   
    elif year in [2017]:
        df = df.drop([0], axis=0).rename(columns={'ESTIMATIVAS DA POPULAÇÃO RESIDENTE NOS MUNICÍPIOS BRASILEIROS COM DATA DE REFERÊNCIA EM 1º DE JULHO DE 2017': 'uf', 'Unnamed: 1': 'uf_id', 'Unnamed: 2': 'mun_id', 'Unnamed: 3': 'nome', 'Unnamed: 4': 'populacao'}).reset_index(drop=True)
        
    else:
        df = df.drop([1], axis=0).rename(columns={'Unnamed: 0': 'uf', 'Unnamed: 1': 'uf_id', 'Unnamed: 2': 'mun_id', 'Unnamed: 3': 'nome', 'Unnamed: 4': 'populacao'}).reset_index(drop=True)
    
    df['id'] = make_id(df)
    
    df['populacao'] = df['populacao'].apply(preprocess_population)
    
    population_tables[year] = df.set_index('id')

100%|██████████| 5/5 [00:02<00:00,  1.72it/s]


In [174]:
for year in years:
    
    print(year, ':', population_tables[year].shape)

2013 : (5570, 5)
2014 : (5570, 5)
2015 : (5570, 5)
2016 : (5570, 5)
2017 : (5570, 5)


In [175]:
%%time

for year in years:
    
    populations = []

    for index in municipios.index:

        municipio = municipios.loc[index]

        id_ = municipio['id']
        
        populations.append(population_tables[year].loc[id_]['populacao'])
        
    municipios[f'pop_{year}'] = populations

municipios.head()

Wall time: 6.69 s


,id,nome,uf_nome,uf_id,uf,regiao,latitude,longitude,pop_2013,pop_2014,pop_2015,pop_2016,pop_2017
0,1100015,Alta Floresta D'Oeste,Rondônia,11,RO,Norte,-11.929692,-61.996128,25728,25652,25578,25506,25437
1,1100023,Ariquemes,Rondônia,11,RO,Norte,-9.907652,-63.033069,101269,102860,104401,105896,107345
2,1100031,Cabixi,Rondônia,11,RO,Norte,-13.505215,-60.677357,6495,6424,6355,6289,6224
3,1100049,Cacoal,Rondônia,11,RO,Norte,-11.433300,-61.442050,85863,86556,87226,87877,88507
4,1100056,Cerejeiras,Rondônia,11,RO,Norte,-13.183969,-61.204778,18041,18013,17986,17959,17934


### Colunas

- id: Código do município (IBGE)
- nome: Nome do município (IBGE)
- uf: UF do município (IBGE)
- uf_id: Código do UF do município (IBGE)
- uf_name: Nome do UF do município (IBGE)
- regiao: Região do município (IBGE)
- pop_(2013, 2017): Populacao estimada do ano no municipio (IBGE)
- latitude: Latitude aproximada do município (Geopy)
- longitude: Longitude aproximada do município (Geopy)

In [176]:
municipios.to_csv('./../data/municipios.csv', index=False)

df = pd.read_csv('./../data/municipios.csv')
df.head()

,id,nome,uf_nome,uf_id,uf,regiao,latitude,longitude,pop_2013,pop_2014,pop_2015,pop_2016,pop_2017
0,1100015,Alta Floresta D'Oeste,Rondônia,11,RO,Norte,-11.929692,-61.996128,25728,25652,25578,25506,25437
1,1100023,Ariquemes,Rondônia,11,RO,Norte,-9.907652,-63.033069,101269,102860,104401,105896,107345
2,1100031,Cabixi,Rondônia,11,RO,Norte,-13.505215,-60.677357,6495,6424,6355,6289,6224
3,1100049,Cacoal,Rondônia,11,RO,Norte,-11.433300,-61.442050,85863,86556,87226,87877,88507
4,1100056,Cerejeiras,Rondônia,11,RO,Norte,-13.183969,-61.204778,18041,18013,17986,17959,17934


### Corrigindo dados faltantes

In [184]:
municipios[np.isnan(df['latitude'])]

,id,nome,uf_nome,uf_id,uf,regiao,latitude,longitude,pop_2013,pop_2014,pop_2015,pop_2016,pop_2017
1752,2800100,Amparo do São Francisco,Sergipe,28,SE,Nordeste,NaN,NaN,2358,2366,2374,2382,2390


In [193]:
# Amparo do São Francisco, Sergipe: https://www.google.com/maps/place/Amparo+de+S%C3%A3o+Francisco+-+SE/@-10.1399259,-36.9294471,15.75z/data=!4m5!3m4!1s0x705a2bfd0a2445d:0x20559e7f2bde1045!8m2!3d-10.1312124!4d-36.9274161

lat = -10.130619
lng = -36.926104

municipios.at[1752, 'latitude']  = lat
municipios.at[1752, 'longitude'] = lng

pd.DataFrame(municipios.loc[1752]).T

,id,nome,uf_nome,uf_id,uf,regiao,latitude,longitude,pop_2013,pop_2014,pop_2015,pop_2016,pop_2017
1752,2800100,Amparo do São Francisco,Sergipe,28,SE,Nordeste,-10.1306,-36.9261,2358,2366,2374,2382,2390


id                           2800100
nome         Amparo do São Francisco
uf_nome                      Sergipe
uf_id                             28
uf                                SE
regiao                      Nordeste
latitude                    -10.1306
longitude                   -36.9261
pop_2013                        2358
pop_2014                        2366
pop_2015                        2374
pop_2016                        2382
pop_2017                        2390
Name: 1752, dtype: object